In [1]:
!pip install -q langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
import os
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="<API_KEY>"
os.environ["LANGCHAIN_PROJECT"]="RAG_ADVANCED"
os.environ['OPENAI_API_KEY'] = "<API_KEY>"


from langsmith import utils
utils.tracing_is_enabled()

True

In [3]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [4]:
#Load Documents
loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/", ),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [6]:
#Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

#Embed
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")

vectorestore = Chroma.from_documents(documents=splits, embedding = embedding, persist_directory="./db004")

#Retriever
retriever = vectorestore.as_retriever()

In [7]:
from langchain.prompts import ChatPromptTemplate

#Decomposition
template = """You are a helful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub questions that can be answers in isolation. \n
Generate multiple search queries related to:{question}\n
Output (3 queries):"""

prompt_decomposition = ChatPromptTemplate.from_template(template)

In [9]:
#LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

#Chain
generate_queries_decomposition = (
    prompt_decomposition
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

#Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})

In [10]:
questions

['1. What are the key components of a large language model (LLM) used in autonomous agent systems?',
 '2. How do perception and sensor integration function within an LLM-powered autonomous agent system?',
 '3. What role does decision-making and planning play in the architecture of an LLM-based autonomous agent?']

In [11]:
#Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question:

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question : \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)


In [13]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
  """Format Q and A pair"""

  formatted_string = ""
  formatted_string += f"Question: {question} \nAnswer:{answer}\n\n"
  return formatted_string.strip()

#LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

q_a_pairs = ""

for q in questions:
  rag_chain = (
      {"context":itemgetter("question") | retriever,
       "question":itemgetter("question"),
       "q_a_pairs":itemgetter("q_a_pairs")}
      | decomposition_prompt
      | llm
      | StrOutputParser()
  )
  answer = rag_chain.invoke({"question":q, "q_a_pairs":q_a_pairs})
  q_a_pair = format_qa_pair(q, answer)
  q_a_pairs = q_a_pair + "\n---\n" + q_a_pair



In [14]:
answer

"In the architecture of an LLM-based autonomous agent, decision-making and planning are fundamental components that enable the agent to operate effectively and autonomously in dynamic environments. Here’s how these elements function within the system:\n\n1. **Decision-Making**: The LLM serves as the core decision-making engine of the autonomous agent. It processes sensory data and contextual information to generate responses and determine the best course of action. The LLM utilizes its understanding of language and context to evaluate different options, weigh potential outcomes, and make informed decisions that align with the agent's goals. This capability is crucial for adapting to real-time changes in the environment and responding appropriately to various situations.\n\n2. **Planning**: Planning involves the formulation of a sequence of actions that the agent needs to take to achieve specific objectives. In LLM-based systems, planning can be enhanced through various mechanisms:\n   